In [1]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension()
import hvplot.pandas

In [4]:
data = pd.read_csv('Derived/All-Journals-Cleaned.csv')

In [21]:

unique_values = pd.unique(data[['JEL1', 'JEL2', 'JEL3', 'JEL4', 'JEL5', 'JEL6', 'JEL7']].values.ravel('K'))

# Remove NaN values from the list
unique_values_cleaned = [value for value in unique_values if pd.notna(value)]

# Print or save the cleaned list of unique values
unique_values_sorted = sorted(unique_values_cleaned)

multi_choice = pn.widgets.MultiChoice(name='MultiSelect', options=unique_values_sorted)

# Global variable to store the selection
selected_options = []

# Callback function to update the global variable and print the selection
def update_selected_options(event):
    global selected_options
    selected_options = event.new
    print("Current Selection:", selected_options)

# Attach the callback to the 'value' parameter of the MultiChoice widget
multi_choice.param.watch(update_selected_options, 'value')

# Create the Panel layout
layout = pn.Column(multi_choice, height=500)

# Display the layout
layout

Column(height=500)
    [0] MultiChoice(name='MultiSelect', options=['A10', 'A11', ...])